In [1]:
# import os

# os.chdir('/Documents/CBA_Stats')

import pandas as pd
import re
from cbastats import DBHelper
import datetime
import pytz

# from cbastats import Scraper
from cbastats import ScraperMongo
from cbastats.Team import *
from cbastats.Player import *
from tqdm.notebook import tqdm
import json

# Scrape Schedule

In [2]:
sina_scraper = ScraperMongo.SinaScraper(ScraperMongo.SINA_SCHEDULE_BASE_URL,ScraperMongo.ENCODING, ScraperMongo.PARSER, ScraperMongo.HEADERS)

In [3]:
sina_scraper.scraper_params

{'qleagueid': {'20-21': '206',
  '19-20': '205',
  '18-19': '198',
  '17-18': '189',
  '16-17': '180',
  '15-16': '171',
  '14-15': '158',
  '13-14': '136',
  '12-13': '107',
  '11-12': '83',
  '10-11': '69',
  '09-10': '56',
  '08-09': '44',
  '07-08': '9',
  '06-07': '2',
  '05-06': '1'},
 'qmonth': {'全部': '',
  '11': '11',
  '12': '12',
  '01': '01',
  '02': '02',
  '03': '03',
  '04': '04',
  '05': '05',
  '06': '06',
  '07': '07',
  '08': '08',
  '09': '09',
  '10': '10'},
 'qteamid': {'全部': '',
  '广东': '1',
  '江苏': '2',
  '上海': '4',
  '浙江': '6',
  '福建': '7',
  '深圳': '8',
  '辽宁': '9',
  '北京': '10',
  '新疆': '11',
  '吉林': '12',
  '广州': '13',
  '山西': '14',
  '山东': '15',
  '广厦': '16',
  '天津': '113',
  '青岛': '114',
  '四川': '182',
  '同曦': '368',
  '北控': '369'}}

In [54]:
seasons_to_scrape = [key for key in sina_scraper.scraper_params['qleagueid']]
scraped_schedule=[]
FULL_REFRESH = False
for season_to_scrape in tqdm(seasons_to_scrape):
    if FULL_REFRESH:
        print(f'You have 10 seconds to stop a full refresh of {len(seasons_to_scrape)} seasons.')
        time.sleep(10)
    else:
        print(f'Not full refresh. Only scraping current season: {sina_scraper.current_season}')
        scraped_schedule.extend(sina_scraper.scrape_schedule(season=sina_scraper.current_season))
        break
    if season_to_scrape == '05-06':
        print('skip 05-06 season - 无上场时间数据')
        continue
    scraped_schedule.extend(sina_scraper.scrape_schedule(season=season_to_scrape))
    ScraperMongo.time.sleep(5)

len(scraped_schedule)

Not full refresh. Only scraping current season: 20-21


351

In [55]:
scraped_schedule[0]

{'轮次': '第1轮',
 '日期': Timestamp('2020-10-17 16:00:00+0800', tz='Asia/Shanghai'),
 '主队': '江苏',
 '比分': '94:102',
 '客队': '青岛',
 '战报': '战报',
 '统计': '统计',
 '组图': '组图',
 '地点': '诸暨',
 '电视': '',
 '轮次_link': '',
 '日期_link': '',
 '主队_link': 'http://cba.sports.sina.com.cn/cba/team/show/2/',
 '比分_link': 'http://cba.sports.sina.com.cn/cba/schedule/show/19143/',
 '客队_link': 'http://cba.sports.sina.com.cn/cba/team/show/114/',
 '战报_link': 'https://sports.sina.com.cn/basketball/cba/2020-10-17/doc-iiznezxr6527423.shtml',
 '统计_link': 'http://cba.sports.sina.com.cn/cba/schedule/show/19143/',
 '组图_link': 'http://slide.sports.sina.com.cn/cba/slide_2_792_255844.html',
 '地点_link': '',
 '电视_link': '',
 'GameID_Sina': '19143',
 '客队ID': '114',
 '主队ID': '2',
 '赛季': '20-21',
 '详细统计': '',
 '比赛回合': ''}

# Connect Database

In [2]:
mongodbio =DBHelper.MongoDBHelper() 
client = mongodbio.create_connection(DBHelper.MONGODB_USERNAME,DBHelper.MONGODB_PWD,DBHelper.MONGODB_ENDPOINT)

db = client['cbaStats']

db.list_collection_names()

existing database ['cbaStats', 'admin', 'local']


['cbaAllGamesBoxScore',
 'cbaGamesStaging',
 'uniqueBoxScores',
 'seasonTeamStats',
 'seasonPlayerStats',
 'cbaGames']

In [3]:
coll_cbaGames = db['cbaGames']
coll_cbaGamesStaging=db['cbaGamesStaging']

# Update games: 

In [58]:
result = mongodbio.insert_new_games(scraped_schedule,coll_cbaGames,coll_cbaGamesStaging)

----------clean up staging collection----------
cbaGamesStaging operation acknowledged!
351 records were deleted from cbaGamesStaging .
cbaGames has 5496 docs.
cbaGamesStaging has 0 docs.
----------insert records into staging collection----------
cbaGamesStaging operation acknowledged!
351 records were inserted into cbaGamesStaging.
----------checking what records to insert into production----------
Production is up-to-date
cbaGames has 5496 docs.


In [59]:
scraped_schedule[0]

{'轮次': '第1轮',
 '日期': Timestamp('2020-10-17 16:00:00+0800', tz='Asia/Shanghai'),
 '主队': '江苏',
 '比分': '94:102',
 '客队': '青岛',
 '战报': '战报',
 '统计': '统计',
 '组图': '组图',
 '地点': '诸暨',
 '电视': '',
 '轮次_link': '',
 '日期_link': '',
 '主队_link': 'http://cba.sports.sina.com.cn/cba/team/show/2/',
 '比分_link': 'http://cba.sports.sina.com.cn/cba/schedule/show/19143/',
 '客队_link': 'http://cba.sports.sina.com.cn/cba/team/show/114/',
 '战报_link': 'https://sports.sina.com.cn/basketball/cba/2020-10-17/doc-iiznezxr6527423.shtml',
 '统计_link': 'http://cba.sports.sina.com.cn/cba/schedule/show/19143/',
 '组图_link': 'http://slide.sports.sina.com.cn/cba/slide_2_792_255844.html',
 '地点_link': '',
 '电视_link': '',
 'GameID_Sina': '19143',
 '客队ID': '114',
 '主队ID': '2',
 '赛季': '20-21',
 '详细统计': '',
 '比赛回合': '',
 '_id': ObjectId('600c7f1bf78ef9ab030126c1')}

In [10]:
current_time = datetime.datetime.now()
timezone_est = pytz.timezone('US/Eastern')
timezone_utc = pytz.timezone('UTC')
current_time = timezone_utc.localize(current_time)
current_time

datetime.datetime(2021, 1, 23, 15, 1, 0, 934453, tzinfo=<UTC>)

### Update game final score

score may be not final when schedule is updated

In [60]:
# update score
# if there is a change to the score, everything in that document will be reset
results=[]
for game in scraped_schedule:
    game.pop('_id',None)
    result=coll_cbaGames.find_one_and_update(
        filter={"$and":[{'GameID_Sina':game['GameID_Sina']},
                        {'比分':{"$ne":game['比分']}}
                        ]},
        update={'$set': game})
    if result:
        results.append(result)

In [61]:
len(results)

0

# Update Game Stats within each game

TODO: put updated games stats into coll_cbaAllGamesBoxScore = db['cbaAllGamesBoxScore'] -> get unique GameID_Sina, delete those (just in case there are duplicates) -> add new game stats

In [13]:
# only updates games that was started before this moment
schedule_to_scrape=mongodbio.select_records(coll_cbaGames,
                                            filter={'详细统计':'',
                                                   '比分':{"$ne":'VS'}})

In [14]:
len(schedule_to_scrape)

5

In [15]:
games_stats, bad_scrapes= sina_scraper.scrape_games(schedule_to_scrape)
if bad_scrapes:
    raise Exception('need to fix bad_scrapes')
else:
    pass

100%|██████████| 5/5 [00:11<00:00,  2.23s/it]


In [16]:
# put game_stats into a dictionary
game_stats_dict={}
unique_gameids = list(games_stats['GameID_Sina'].value_counts().index)
for gameid in unique_gameids:
    game_stats_dict[gameid] = games_stats.loc[games_stats['GameID_Sina']==gameid].to_dict('record')

# update the stats in mongo
results=[]
for game_id, game_stats in game_stats_dict.items():
    print(game_id)
    results.append(coll_cbaGames.update_one({'GameID_Sina':game_id},{"$set":{"详细统计":game_stats}}))

19684
19683
19686
19685
19682


### Manual Fix

# Insert new stats into cbaAllGamesBoxScore

### Insert into coll_cbaAllGamesBoxScore = db['cbaAllGamesBoxScore']

In [17]:
coll_cbaAllGamesBoxScore = db['cbaAllGamesBoxScore']

In [18]:
for game_id, game_stats in game_stats_dict.items():
    print(game_id)
    mongodbio.delete_records(coll_cbaAllGamesBoxScore,{'GameID_Sina':game_id})
    mongodbio.insert_records(coll_cbaAllGamesBoxScore,game_stats)

19684
cbaAllGamesBoxScore operation acknowledged!
0 records were deleted from cbaAllGamesBoxScore .
cbaAllGamesBoxScore operation acknowledged!
45 records were inserted into cbaAllGamesBoxScore.
19683
cbaAllGamesBoxScore operation acknowledged!
0 records were deleted from cbaAllGamesBoxScore .
cbaAllGamesBoxScore operation acknowledged!
39 records were inserted into cbaAllGamesBoxScore.
19686
cbaAllGamesBoxScore operation acknowledged!
0 records were deleted from cbaAllGamesBoxScore .
cbaAllGamesBoxScore operation acknowledged!
39 records were inserted into cbaAllGamesBoxScore.
19685
cbaAllGamesBoxScore operation acknowledged!
0 records were deleted from cbaAllGamesBoxScore .
cbaAllGamesBoxScore operation acknowledged!
39 records were inserted into cbaAllGamesBoxScore.
19682
cbaAllGamesBoxScore operation acknowledged!
37 records were deleted from cbaAllGamesBoxScore .
cbaAllGamesBoxScore operation acknowledged!
37 records were inserted into cbaAllGamesBoxScore.


# use cbaAllGamesBoxScore to update unique boxscore (full update)

TODO: partial update? use new game_stats to update unique boxscore

TODO: add a field: insert time, to determine new boxscore

In [19]:
coll_cbaAllGamesBoxScore = db['cbaAllGamesBoxScore']

all_box_scores=pd.DataFrame(mongodbio.select_records(coll_cbaAllGamesBoxScore,{}))
all_box_scores.shape

(137846, 33)

In [20]:
simple_boxscores = all_box_scores[['篮板', '助攻','抢断','盖帽','得分']].copy()

uniq_simple_boxscores = simple_boxscores.drop_duplicates().sort_values(by='得分',ascending=False).reset_index(drop=True)

uniq_simple_boxscores = uniq_simple_boxscores.reset_index()

#删除所有数据统计为0 的数据
uniq_simple_boxscores.drop(uniq_simple_boxscores.loc[uniq_simple_boxscores[['篮板', '助攻', '抢断', '盖帽', '得分']].sum(axis=1)==0].index,inplace=True)

unique_boxscores = uniq_simple_boxscores.merge(all_box_scores,on=['篮板', '助攻', '抢断', '盖帽', '得分'])
unique_boxscores.head()

,index,篮板,助攻,抢断,盖帽,得分,_id,球员_link,球员,号码,...,对手ID,球队,对手,地点,2分中,2分投,3分中,3分投,罚球中,罚球投
0,0,10.0,4.0,2.0,0.0,82.0,600a58a14d892ff35d137944,http://cba.sports.sina.com.cn/cba/player/show/...,麦克鲁姆,3,...,1,浙江,广东,,21.0,35.0,5.0,14.0,25.0,27.0
1,1,1.0,5.0,8.0,0.0,75.0,600a58b74d892ff35d1402b1,http://cba.sports.sina.com.cn/cba/player/show/...,昆西杜比,5,...,14,浙江,山西,义乌,16.0,23.0,9.0,15.0,16.0,20.0
2,2,8.0,7.0,4.0,0.0,75.0,600a585d4d892ff35d1236c0,http://cba.sports.sina.com.cn/cba/player/show/...,弗雷戴特,32,...,369,上海,北控,北京,17.0,24.0,7.0,10.0,20.0,21.0
3,3,10.0,4.0,1.0,0.0,74.0,600a58ab4d892ff35d13b850,http://cba.sports.sina.com.cn/cba/player/show/...,鲍比-布朗,6,...,182,东莞,四川,成都 四川省体育馆,21.0,37.0,5.0,15.0,17.0,20.0
4,4,4.0,2.0,1.0,0.0,74.0,600a58884d892ff35d1333e5,http://cba.sports.sina.com.cn/cba/player/show/...,康宁汉姆,6,...,7,同曦,福建,,15.0,24.0,7.0,14.0,23.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106423,29136,4.0,2.0,1.0,0.0,0.0,600a58a74d892ff35d139618,http://cba.sports.sina.com.cn/cba/player/show/...,陈江华,6,...,4,广东,上海,,0.0,3.0,0.0,3.0,0.0,0.0
106424,29136,4.0,2.0,1.0,0.0,0.0,600a58ac4d892ff35d13c467,http://cba.sports.sina.com.cn/cba/player/show/...,张伟,24,...,10,广厦,北京,,0.0,1.0,0.0,1.0,0.0,0.0
106425,29136,4.0,2.0,1.0,0.0,0.0,600a58c04d892ff35d1426ca,http://cba.sports.sina.com.cn/cba/player/show/...,解立彬,7,...,8,北京,东莞,,0.0,2.0,0.0,1.0,0.0,0.0
106426,29136,4.0,2.0,1.0,0.0,0.0,600a58c04d892ff35d1430f6,http://cba.sports.sina.com.cn/cba/player/show/...,郭磊,12,...,7,山东,福建,,0.0,1.0,0.0,0.0,0.0,0.0


In [24]:
occurrences = uniq_simple_boxscores.to_dict('record')

for row in tqdm(occurrences):
    row['事件'] = unique_boxscores.loc[unique_boxscores['index']==row['index']].to_dict('record')

In [38]:
occurrences[0]

{'index': 0,
 '篮板': 10.0,
 '助攻': 4.0,
 '抢断': 2.0,
 '盖帽': 0.0,
 '得分': 82.0,
 '事件': [{'index': 0,
   '篮板': 10.0,
   '助攻': 4.0,
   '抢断': 2.0,
   '盖帽': 0.0,
   '得分': 82.0,
   '_id': ObjectId('600a58a14d892ff35d137944'),
   '球员_link': 'http://cba.sports.sina.com.cn/cba/player/show/4834/',
   '球员': '麦克鲁姆',
   '号码': '3',
   '出场时间': 48.0,
   '首发': 1.0,
   '进攻篮板': 2.0,
   '防守篮板': 8.0,
   '犯规': 5.0,
   '失误': 5.0,
   '扣篮': 1.0,
   '被侵': 20.0,
   '快攻': '',
   '轮次': '第37轮',
   '日期': Timestamp('2015-01-30 11:35:00'),
   '赛季': '14-15',
   'GameID_Sina': '13056',
   '球队ID': '6',
   '对手ID': '1',
   '球队': '浙江',
   '对手': '广东',
   '地点': '',
   '2分中': 21.0,
   '2分投': 35.0,
   '3分中': 5.0,
   '3分投': 14.0,
   '罚球中': 25.0,
   '罚球投': 27.0}],
 '_id': ObjectId('600c3bdbf78ef9ab0300b4ef')}

In [25]:
def batch_list(lst:list, batch_size=1):
    l = len(lst)
    for ndx in range(0, l, batch_size):
        yield lst[ndx:min(ndx + batch_size, l)]

coll_uniqueBoxScores = db['uniqueBoxScores']
mongodbio.delete_records(coll_uniqueBoxScores,{})
bad_batch = []
for batch in tqdm(batch_list(occurrences,1000)):
    try:
        mongodbio.insert_records(coll_uniqueBoxScores,batch)
    except:
        bad_batch.extend(batch)

uniqueBoxScores operation acknowledged!
29120 records were deleted from uniqueBoxScores .


uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores operation acknowledged!
1000 records were inserted into uniqueBoxScores.
uniqueBoxScores opera

# 从新数据中找出罕见数据

In [50]:
box_score_item=['篮板','助攻','抢断','盖帽','得分']
field_to_select = {item:1 for item in box_score_item}

In [49]:
coll_uniqueBoxScores = db['uniqueBoxScores']
# coll_uniqueBoxScores.find_one({})

def print_rare_stats(coll_uniqueBoxScores,player_stats:dict):
    # TODO: print上次出现时间或上次出现的球员
    # srasb:score,rebound,assists,steal,block
    srasb=coll_uniqueBoxScores.count_documents({'篮板': {"$gte":player_stats['篮板']},
                                              '助攻': {"$gte":player_stats['助攻']},
                                              '抢断': {"$gte":player_stats['抢断']},
                                              '盖帽': {"$gte":player_stats['盖帽']},
                                              '得分': {"$gte":player_stats['得分']}})
    srab=coll_uniqueBoxScores.count_documents({'篮板': {"$gte":player_stats['篮板']},
                                              '助攻': {"$gte":player_stats['助攻']},
                                              '盖帽': {"$gte":player_stats['盖帽']},
                                              '得分': {"$gte":player_stats['得分']}})
    sras=coll_uniqueBoxScores.count_documents({'篮板': {"$gte":player_stats['篮板']},
                                              '助攻': {"$gte":player_stats['助攻']},
                                              '盖帽': {"$gte":player_stats['抢断']},
                                              '得分': {"$gte":player_stats['得分']}})
    sra=coll_uniqueBoxScores.count_documents({'篮板': {"$gte":player_stats['篮板']},
                                              '助攻': {"$gte":player_stats['助攻']},
                                              '得分': {"$gte":player_stats['得分']}})
    if (srasb<=10)|(srab<=10)|(sras<=10)|(sra<=10):
        print('------------------------------------------------')
        print(f"{player_stats['赛季']}赛季,{player_stats['日期']}，{player_stats['球队']}的{player_stats['球员']}，得分 {player_stats['得分']}，篮板 {player_stats['篮板']}，助攻 {player_stats['助攻']}，抢断 {player_stats['抢断']}，盖帽 {player_stats['盖帽']}")
        print(f"得分，篮板，助攻，抢断，盖帽->历史上第{srasb}次出现")
        print(f"得分，篮板，助攻，抢断->历史上第{sras}次出现")
        print(f"得分，篮板，助攻，盖帽->历史上第{srab}次出现")
        print(f"得分，篮板，助攻->历史上第{sra}次出现")
        print('------------------------------------------------')
        

rare_stats=[]
for game_id, game_stats in tqdm(game_stats_dict.items(),unit='game'):
    print(f"checking game: {game_id}")
    for player_stats in tqdm(game_stats,unit='player stats'):
        if sum([player_stats[item] for item in box_score_item])==0:
            continue
        print_rare_stats(coll_uniqueBoxScores,player_stats)
        rare_stats.append(player_stats)

checking game: 19684



checking game: 19683



checking game: 19686



checking game: 19685



checking game: 19682


# 数据历史排行榜

新浪数据不全，直接从官方scrape

# Update Calculated Stats

### Get GameStats

In [87]:
sina_scraper.current_season

'20-21'

In [88]:
# pull data 
all_data = mongodbio.select_records(coll_cbaGames,filter={'详细统计':{"$ne":''},'赛季':sina_scraper.current_season})

# convert to a pandas dataframe
all_games_stats = []
for game in all_data:
    for row in game['详细统计']:
        all_games_stats.append(row)

df_all_games_stats=pd.DataFrame(all_games_stats)

In [89]:
df_all_games_stats.columns

Index(['球员_link', '球员', '号码', '出场时间', '首发', '进攻篮板', '防守篮板', '助攻', '犯规', '抢断',
       '失误', '盖帽', '扣篮', '被侵', '快攻', '得分', '轮次', '日期', '赛季', 'GameID_Sina',
       '球队ID', '对手ID', '球队', '对手', '地点', '2分中', '2分投', '3分中', '3分投', '罚球中',
       '罚球投', '篮板'],
      dtype='object')

### Calculate Team Stats

In [90]:
teams = Team('',df_all_games_stats)

teams_df = pd.concat([teams.mov, teams.tm_pace, teams.tm_ortg, teams.tm_drtg, teams.tm_nrtg], axis=1)

teams_df.columns=['场均净胜分MOV', 'Pace', 'OffensiveRating', 'DefensiveRating', 'NetRating']
teams_df = teams_df.sort_values(by='NetRating',ascending=False)
teams_df.round(1)

,场均净胜分MOV,Pace,OffensiveRating,DefensiveRating,NetRating
球队,,,,,
辽宁,14.4,93.8,124.7,109.0,15.6
广东,14.8,102.3,123.0,108.2,14.8
浙江,10.4,94.9,119.4,108.3,11.1
新疆,5.0,94.9,115.9,110.5,5.4
山东,4.0,94.7,113.4,109.2,4.3
广厦,2.3,95.1,118.1,115.7,2.5
吉林,1.9,91.2,115.6,113.4,2.2
青岛,1.9,95.3,112.1,110.1,2.0
上海,0.7,93.6,115.8,115.0,0.8


### Truncate and Insert Curent Season Team Stats

In [91]:
db.list_collection_names()

['cbaAllGamesBoxScore',
 'cbaGamesStaging',
 'uniqueBoxScores',
 'seasonTeamStats',
 'seasonPlayerStats',
 'cbaGames']

In [92]:
coll_teamStats = db['seasonTeamStats']

# insert
mongodbio.delete_records(coll_teamStats,{})
mongodbio.insert_records(coll_teamStats,teams_df.to_dict('record'))

seasonTeamStats operation acknowledged!
19 records were deleted from seasonTeamStats .
seasonTeamStats operation acknowledged!
19 records were inserted into seasonTeamStats.


### Calculate Player Stats

In [93]:
players = Player('',df_all_games_stats)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
players_df = pd.concat([players.plr_ortg, players.plr_drtg, players.plr_nrtg,players.plr_usg], axis=1)

players_df.columns=['OffensiveRating', 'DefensiveRating', 'NetRating','UsagePercent']
players_df = players_df.sort_values(by=['NetRating','UsagePercent'],ascending=False)


players_df = pd.merge(players_df,players.plr_total_stats,left_index=True,right_index=True)
players_df.reset_index(inplace=True)
players_df.sort_values(by=['球队','NetRating','UsagePercent'],ascending=False,inplace=True)
players_df

,球员,球队,OffensiveRating,DefensiveRating,NetRating,UsagePercent,出场时间,首发,进攻篮板,防守篮板,...,被侵,得分,2分中,2分投,3分中,3分投,罚球中,罚球投,篮板,出场
266,约翰逊,青岛,116.918434,111.932798,4.985636,30.504621,380,7,63,116,...,118,288,105,162,0,0,78,151,179,15
39,刘传兴,青岛,117.669163,114.691009,2.978154,15.277952,628,18,78,143,...,88,230,79,131,0,0,72,102,221,29
250,王睿泽,青岛,117.487166,115.428752,2.058414,18.543895,189,5,6,18,...,6,93,17,31,19,39,2,3,24,9
158,斯蒂马克,青岛,114.936632,116.047588,-1.110956,28.342506,126,0,26,45,...,22,84,32,58,3,6,11,25,71,9
318,赵泰隆,青岛,113.728280,116.071007,-2.342726,14.135525,446,16,6,25,...,16,156,18,34,37,96,9,17,31,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,朱瀛,上海,NaN,124.078772,NaN,19.608027,2,0,1,1,...,1,1,0,0,0,0,1,2,2,1
375,黄旭,上海,NaN,120.851037,NaN,14.483202,40,0,1,1,...,1,17,4,6,3,7,0,0,2,6
82,周彦旭,上海,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
242,王旭,上海,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Truncate and Insert Current Season Player Stats

In [94]:
coll_playerStats = db['seasonPlayerStats']

# insert
mongodbio.delete_records(coll_playerStats,{})
mongodbio.insert_records(coll_playerStats,players_df.to_dict('record'))

seasonPlayerStats operation acknowledged!
380 records were deleted from seasonPlayerStats .
seasonPlayerStats operation acknowledged!
380 records were inserted into seasonPlayerStats.


# Update play by play